In [8]:
load_from_disk = True
import numpy as np
import cv2
import time

In [10]:
if load_from_disk:
    penval = np.load('penval_red.npy')
    
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
canvas = None
kernal = np.ones((5,5), np.uint8)
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
noiseth = 600

wipethresh = 40000
x1, y1 = 0, 0
clear = False
while(True):
    img_og = cap.read()[1]
    img_og = cv2.flip(img_og, 1)
    if canvas is None:
        canvas = np.zeros_like(img_og)
 

    
    hsv = cv2.cvtColor(img_og, cv2.COLOR_BGR2HSV)
    
    lower_range = penval[0]
    upper_range = penval[1]
    
    mask = cv2.inRange(hsv, lower_range, upper_range)
    mask = cv2.erode(mask, kernal, iterations = 1)
    mask = cv2.dilate(mask, kernal, iterations = 2)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    
    if contours and cv2.contourArea(max(contours, key = cv2.contourArea)) > noiseth:
        c = max(contours, key = cv2.contourArea)
        x2,y2,w,h = cv2.boundingRect(c)
        area = cv2.contourArea(c)
        if x1 == 0 and y1 ==0:
            x1, y1 = x2, y2
        else: 
            canvas = cv2.line(canvas, (x1,y1),(x2,y2),[0, 0, 255],5)
            
        x1, y1 = x2, y2
        if area > wipethresh:
            clear = True
        #cv2.rectangle(img_og, (x2,y2), (x2+w,y2+h), (0,25,255),2)
    else:
        x1, y1 = 0, 0
    #_, mask = cv2.threshold(cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY),20, 255, cv2.THRESH_BINARY)
    #fore = cv2.bitwise_and(canvas, canvas, mask = mask)
    #back = cv2.bitwise_and(img_og, img_og, mask= cv2.bitwise_not(mask))
    img_og = cv2.add(img_og, canvas)
    #res = cv2.bitwise_and(img_og, img_og, mask = mask)
    #mask_2 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    
    #stacked = np.hstack((mask_2,img_og, res ))
    #stacked = cv2.resize(stacked, None, fx = 0.4,fy=0.4)
    cv2.imshow("wtf", img_og)
    key = cv2.waitKey(1)
    
    if key == 27:
        break
    if key == ord('c'):
        time.sleep(2)
        canvas = None
        clear = False
cv2.destroyAllWindows()
cap.release()